**📌 Modèle RNN/LSTM pour générer un résumé des commentaires d'une description** \
Nous allons utiliser un modèle de type Séquence-à-Séquence (Seq2Seq) avec un encodeur-décodeur LSTM pour générer des résumés des commentaires associés à chaque description. \

**✅ 1. Modèle recommandé : LSTM avec Attention** \
Les RNN classiques ont des limites (perte de contexte sur de longues séquences), donc nous allons améliorer le modèle en ajoutant un mécanisme d'attention. \

**Le modèle Seq2Seq avec Attention est utilisé pour :** \

Encoder plusieurs commentaires d'une description dans un LSTM.
Décoder ces informations en générant un résumé pertinent.


In [4]:
!pip install tensorflow


In [2]:
!pip install tensorflow-gpu


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import pandas as pd

In [3]:
import pandas as pd

merged_data = pd.read_excel("/content/merged_data_english_only (1).xlsx")



In [5]:
!pip install -q openpyxl tqdm


# **LSTM**

## 🔁 Modèle LSTM Seq2Seq avec Attention — Interprétation du Code

Ce modèle permet de **générer des descriptions synthétiques** à partir d’un ensemble de **commentaires clients concaténés**. Il suit une architecture **encoder-decoder avec mécanisme d’attention**, typique des tâches de résumé.

---

### 🧹 Prétraitement
- Nettoyage des commentaires et descriptions (suppression HTML, ponctuation, etc.).
- Concaténation des commentaires pour chaque logement → un seul texte long.
- Ajout des tokens spéciaux `<start>` et `<end>` dans les descriptions cibles pour le décodeur.

---

### ✂️ Tokenisation & Padding
- Utilisation de `Tokenizer` pour convertir les textes en séquences d’index.
- Tronquage/padding à des longueurs fixes :
  - `MAX_INPUT_LENGTH = 50` pour les commentaires
  - `MAX_OUTPUT_LENGTH = 20` pour les descriptions

---

### 🧠 Architecture du Modèle

**Encodeur :**
- Embedding → LSTM → récupère les états `state_h`, `state_c`
- Produit une séquence encodée + mémoire de contexte

**Décodeur :**
- Embedding → LSTM initialisé avec les états de l’encodeur
- Produit une séquence de sortie mot par mot

**Attention :**
- `Dot` product entre chaque sortie du décodeur et chaque état de l’encodeur
- Softmax sur ces scores pour obtenir les **poids d’attention**
- Pondération des états de l’encodeur → vecteur de contexte
- Concaténation avec la sortie du LSTM pour enrichir l’information

---

### ⚙️ Entraînement
- Fonction de perte : `sparse_categorical_crossentropy`
- Optimiseur : `adam`
- `EarlyStopping` pour éviter l’overfitting
- `Decoder Target` : les séquences cibles sont décalées d’un pas (teacher forcing)

---

### 🔍 Inference (Génération)
- Création de modèles **d’encodage et décodage séparés**
- Génération **mot par mot** avec prédiction + attention à chaque étape
- Arrêt à la détection du token `<end>`

---



In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Dot, Activation, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
import re

# Nettoyage du texte
def clean_text(text):
    text = re.sub(r'<br\s*/?>', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text.strip()

# Grouper les commentaires par description
df_grouped = merged_data.groupby("description_translated")["comments_translated"].apply(lambda x: ' '.join(x)).reset_index()
df_grouped.columns = ["description", "all_comments"]

# Nettoyer les colonnes
df_grouped['all_comments'] = df_grouped['all_comments'].apply(clean_text)
df_grouped['description'] = df_grouped['description'].apply(lambda x: '<start> ' + clean_text(x) + ' <end>')

# Paramètres
MAX_INPUT_LENGTH = 50  # Taille maximale des commentaires groupés
MAX_OUTPUT_LENGTH = 20  # Longueur du résumé attendu
VOCAB_SIZE = 5000  # Nombre de mots retenus (les plus fréquents)
LSTM_UNITS = 128    # Taille de la mémoire LSTM
EMBEDDING_DIM = 128
BATCH_SIZE = 16
EPOCHS = 50

# Tokenisation
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(df_grouped['all_comments'])
input_seq = tokenizer.texts_to_sequences(df_grouped['all_comments'])
input_padded = pad_sequences(input_seq, maxlen=MAX_INPUT_LENGTH, padding='post')

tokenizer_summary = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer_summary.fit_on_texts(df_grouped['description'])
target_seq = tokenizer_summary.texts_to_sequences(df_grouped['description'])
target_padded = pad_sequences(target_seq, maxlen=MAX_OUTPUT_LENGTH, padding='post')

decoder_target_data = np.zeros_like(target_padded)
decoder_target_data[:, :-1] = target_padded[:, 1:]
decoder_target_data = np.expand_dims(decoder_target_data, -1)

# Modèle
enc_input = Input(shape=(MAX_INPUT_LENGTH,))
enc_emb = Embedding(VOCAB_SIZE, EMBEDDING_DIM, mask_zero=True)(enc_input)
enc_lstm, state_h, state_c = LSTM(LSTM_UNITS, return_sequences=True, return_state=True)(enc_emb)

dec_input = Input(shape=(MAX_OUTPUT_LENGTH,))
dec_emb = Embedding(VOCAB_SIZE, EMBEDDING_DIM, mask_zero=True)(dec_input)
dec_lstm, _, _ = LSTM(LSTM_UNITS, return_sequences=True, return_state=True)(dec_emb, initial_state=[state_h, state_c])

attn_score = Dot(axes=[2, 2])([dec_lstm, enc_lstm])
attn_weights = Activation('softmax')(attn_score)
context = Dot(axes=[2, 1])([attn_weights, enc_lstm])
dec_combined = Concatenate(axis=-1)([context, dec_lstm])
dec_dense = Dense(VOCAB_SIZE, activation='softmax')(dec_combined)

model = Model([enc_input, dec_input], dec_dense)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Entraînement
early_stop = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
model.fit([input_padded, target_padded], decoder_target_data,
          batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stop])

# Modèles d'inférence
encoder_model = Model(enc_input, [enc_lstm, state_h, state_c])

dec_state_input_h = Input(shape=(LSTM_UNITS,))
dec_state_input_c = Input(shape=(LSTM_UNITS,))
enc_out_input = Input(shape=(MAX_INPUT_LENGTH, LSTM_UNITS))
dec_input_infer = Input(shape=(1,))

dec_emb_layer = Embedding(VOCAB_SIZE, EMBEDDING_DIM, mask_zero=True)
dec_emb_infer = dec_emb_layer(dec_input_infer)
dec_lstm_infer, dec_h, dec_c = LSTM(LSTM_UNITS, return_sequences=True, return_state=True)(dec_emb_infer, initial_state=[dec_state_input_h, dec_state_input_c])
attn_score_inf = Dot(axes=[2, 2])([dec_lstm_infer, enc_out_input])
attn_weights_inf = Activation('softmax')(attn_score_inf)
context_inf = Dot(axes=[2, 1])([attn_weights_inf, enc_out_input])
dec_combined_inf = Concatenate(axis=-1)([context_inf, dec_lstm_infer])
dec_out_inf = Dense(VOCAB_SIZE, activation='softmax')(dec_combined_inf)

decoder_model = Model(
    [dec_input_infer, enc_out_input, dec_state_input_h, dec_state_input_c],
    [dec_out_inf, dec_h, dec_c]
)

# Fonction de génération
def generate_summary(text):
    text = clean_text(text)
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=MAX_INPUT_LENGTH, padding='post')
    enc_out, state_h, state_c = encoder_model.predict(padded, verbose=0)

    target_seq = np.zeros((1, 1))
    start_token = tokenizer_summary.word_index.get('<start>', tokenizer_summary.word_index.get('<OOV>', 1))
    target_seq[0, 0] = start_token

    result = []
    for _ in range(MAX_OUTPUT_LENGTH):
        output_tokens, h, c = decoder_model.predict([target_seq, enc_out, state_h, state_c], verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_summary.index_word.get(sampled_token_index, '')

        if sampled_word == '<end>' or sampled_word == '' or sampled_word == '<OOV>':
            break
        result.append(sampled_word)
        target_seq[0, 0] = sampled_token_index
        state_h, state_c = h, c

    return ' '.join(result)

# Génération des résumés
tqdm.pandas(desc="Génération des descriptions")
df_grouped['generated_description'] = df_grouped['all_comments'].progress_apply(generate_summary)

# Export
output_path = "descriptions_from_comments_cleaned_lstm.xlsx"
df_grouped.to_excel(output_path, index=False)
print(f"\u2705 Descriptions générées et sauvegardées dans '{output_path}'")






Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 580ms/step - loss: 8.5061
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 701ms/step - loss: 7.7454
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 444ms/step - loss: 5.8620
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 467ms/step - loss: 5.6629
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 301ms/step - loss: 5.4622
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 296ms/step - loss: 5.3331
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 426ms/step - loss: 5.3098
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 296ms/step - loss: 5.2024
Epoch 9/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 281ms/step - loss: 5.1300
Epoch 10/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 454ms/step - loss: 5.1060
Epoch 11/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 288ms/step - loss: 5.0961
Epoch 12/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 304ms/step - loss: 4.9519
Epoch 13/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 454ms/step - loss: 5.0215
Epoch 14/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 315ms/step - loss: 5.0675
Epoch 15/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 298ms/step - l

Génération des descriptions: 100%|██████████| 150/150 [03:31<00:00,  1.41s/it]


✅ Descriptions générées et sauvegardées dans 'descriptions_from_comments_cleaned_lstm.xlsx'


In [7]:
df_grouped.head()

,description,all_comments,generated_description
0,<start> at home in paris its possible apartmen...,simply put this apartment is excellent everyt...,bank bank bank bank bank bank bank
1,<start> minimum 1 month stay mobility lease ...,very warm welcome from simone nadèges mother w...,bank bank bank bank bank bank bank
2,<start> 180 sweeping view over paris including...,sophie was a very accommodating host answering...,bank bank bank bank bank bank bank
3,<start> in the heart of one of the oldest dist...,alains apartment is located in one of the most...,bank bank bank bank bank bank bank bank
4,<start> 1 to 10month mobility lease only acco...,boris apartment was fantastic large clean and ...,bank bank bank bank bank bank bank bank


## 📘 Passage de LSTM avec attention à BART / T5

### 🧠 Motivation initiale : LSTM + Attention

Dans un premier temps, nous avons construit un modèle **Seq2Seq** basé sur une architecture **LSTM avec attention**. Cette méthode permettait de traiter des séquences de commentaires clients et de générer des descriptions synthétiques. Malgré sa capacité à apprendre un schéma de résumé, nous avons rencontré plusieurs limitations :

- Les **séquences longues** (plusieurs commentaires concaténés) **saturaient rapidement la mémoire** du modèle.
- Le **vocabulaire généré était pauvre et répétitif**, avec des sorties peu informatives (ex. : `bank bank bank...`).
- L’**entraînement nécessitait beaucoup de données** et un **temps de convergence élevé**.
- LSTM reste limité dans la **modélisation du contexte global à long terme**, même avec une couche d’attention.

---

### 🔄 Évolution : Utilisation de modèles Transformer pré-entraînés (T5 / BART)

Pour dépasser ces limitations, nous avons migré vers des modèles **pré-entraînés de type Transformer**, plus particulièrement :

- **T5** (*Text-to-Text Transfer Transformer*)
- **BART** (*Bidirectional and Auto-Regressive Transformers*)

Ces modèles sont **pré-entraînés sur des tâches de résumé**, et peuvent générer des descriptions **cohérentes, informatives et pertinentes**, **sans nécessiter d’entraînement lourd**.

---

### ✅ Avantages de BART et T5

| Modèle | Avantages |
|--------|-----------|
| **BART** | Très performant pour le résumé, bidirectionnel (comprend bien le contexte global), structure auto-régressive. |
| **T5**   | Approche text-to-text unifiée, très efficace pour la traduction, le résumé, la reformulation. |


# **Résumé avec  BART**

## 🤖 Résumé automatique avec BART (facebook/bart-large-cnn)

Ce script utilise le modèle pré-entraîné **BART (Bidirectional and Auto-Regressive Transformer)** pour générer des descriptions réalistes à partir de **commentaires utilisateurs concaténés**.

---

### 🔧 Chargement du modèle
- Le modèle **`facebook/bart-large-cnn`** est spécialement entraîné pour les tâches de **résumé automatique**.
- On utilise la classe `BartTokenizer` pour tokeniser les textes d’entrée, et `BartForConditionalGeneration` pour générer des résumés.

---

### 🧠 Pipeline de traitement
1. **Préparation des données** :
   - Les commentaires associés à une même description sont regroupés (`groupby` sur `description_translated`).
   - Tous les commentaires sont concaténés (`' '.join(x)`), donnant un long paragraphe par annonce.

2. **Résumé** :
   - On applique une fonction `bart_summary()` sur chaque ligne.
   - Cette fonction encode le texte avec le tokenizer puis utilise `model.generate()` pour produire un résumé :
     - `num_beams=4` : beam search pour améliorer la qualité.
     - `length_penalty=2.0` : pénalise les résumés trop longs.
     - `no_repeat_ngram_size=3` : évite les répétitions.
     - `early_stopping=True` : arrête dès que le modèle détecte la fin du résumé.

3. **Sauvegarde** :
   - Les résumés générés sont stockés dans une nouvelle colonne `generated_description`.
   - Le résultat final est exporté dans un fichier Excel.

---

### ✅ Avantages de BART
- Très performant pour les **résumés extractifs et abstratifs**.
- S’adapte bien à des textes informels comme les avis clients.
- Permet de résumer automatiquement **plusieurs commentaires** en une seule description synthétique.

---



In [9]:
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
from tqdm import tqdm

# 🔧 Charger le modèle BART
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# 🔁 Fonction de résumé
def bart_summary(text, max_len=130, min_len=30):
    inputs = tokenizer.batch_encode_plus([text], return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = model.generate(
        inputs['input_ids'],
        num_beams=4,
        length_penalty=2.0,
        max_length=max_len,
        min_length=min_len,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 🧠 Regrouper les commentaires par description
df_grouped = merged_data.groupby("description_translated")["comments_translated"] \
    .apply(lambda x: ' '.join(x)).reset_index()
df_grouped.columns = ["description", "all_comments"]

# 📝 Générer les résumés
tqdm.pandas(desc="📄 Résumé avec BART")
df_grouped["generated_description"] = df_grouped["all_comments"].progress_apply(bart_summary)

# 💾 Sauvegarder dans Excel
df_grouped.to_excel("generated_descriptions_bart.xlsx", index=False)
print("✅ Résumés générés avec BART et enregistrés dans 'generated_descriptions_bart.xlsx'")


📄 Résumé avec BART: 100%|██████████| 150/150 [1:21:54<00:00, 32.77s/it]


✅ Résumés générés avec BART et enregistrés dans 'generated_descriptions_bart.xlsx'


In [ ]:


# 🔹 Installations nécessaires
!pip install -q transformers sentencepiece openpyxl




In [10]:

df_grouped.head()

,description,all_comments,generated_description
0,"""At home in Paris"", it's possible!<br />Apartm...","Simply put, this apartment is excellent. Ever...",Batignolles is a great neighborhood chock full...
1,** Minimum 1 month stay **<br /><br />Mobility...,Very warm welcome from Simone (Nadège's mother...,The accommodation was in accordance with the p...
2,"- 180° sweeping view over Paris, including all...","Sophie was a very accommodating host, answerin...","Sophie was a very accommodating host, answerin..."
3,- In the heart of one of the oldest district o...,Alain's apartment is located in one of the mos...,Alain has put a great deal of heart into the f...
4,"1 to 10-month ""mobility lease"" only - accordin...","Boris' apartment was fantastic. Large, clean, ...","Boris' apartment was fantastic. Large, clean, ..."
